In [1]:
import ast
import json
import pickle
import numpy as np
from pathlib import Path
from sklearn.metrics import classification_report


In [2]:
# Load the pickled list
with open("/Utilisateurs/umushtaq/emotion_analysis_comics/incontext_learning/results/icl_mistral-7b-instruct-v0.3-bnb-4bit/results_10.pickle", 'rb') as f:
    
    results = pickle.load(f)

In [3]:
results

{'ground_truths': [['surprise', 'joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['surprise'],
  ['joy'],
  ['joy'],
  ['neutral'],
  ['neutral'],
  ['neutral'],
  ['neutral'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['neutral'],
  ['sadness'],
  ['sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['joy'],
  ['anger', 'surprise'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['surprise', 'joy'],
  ['fear', 'sadness'],
  ['fear', 'sadness'],
  ['fear', 'surprise'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['fear', 'sadness'],
  ['fear', 'sadness', 'surprise'],
  ['sadness'],
  ['sadness'],
  ['fear', 'sadness'],
  ['sadness', 'surprise'],
  ['sadness', 'surprise'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger', 'disgust'],
  ['joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['surprise', 'joy'],
  ['anger', 'surprise']

In [4]:
grounds = results['ground_truths']

In [5]:
predictions = results['predictions']

In [6]:
len(grounds), len(predictions)

(1776, 1776)

In [7]:
grounds

[['surprise', 'joy'],
 ['joy'],
 ['surprise', 'joy'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['surprise'],
 ['joy'],
 ['joy'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['anger', 'disgust'],
 ['anger', 'disgust'],
 ['neutral'],
 ['sadness'],
 ['sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['fear', 'surprise'],
 ['surprise'],
 ['joy'],
 ['anger', 'surprise'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['surprise', 'joy'],
 ['fear', 'sadness'],
 ['fear', 'sadness'],
 ['fear', 'surprise'],
 ['anger', 'disgust'],
 ['anger', 'disgust'],
 ['anger', 'disgust'],
 ['fear', 'sadness'],
 ['fear', 'sadness', 'surprise'],
 ['sadness'],
 ['sadness'],
 ['fear', 'sadness'],
 ['sadness', 'surprise'],
 ['sadness', 'surprise'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['anger'],
 ['anger', 'disgust'],
 ['joy'],
 ['joy'],
 ['surprise', 'joy'],
 ['surprise', 'joy'],
 ['anger', 'surprise'],
 ['anger', 'surprise'],
 ['neutral'],
 ['joy'],
 ['joy'],
 ['neutral'

In [8]:
predictions

['{"utterance_emotions": "[\'neutral\']"}',
 '{"utterance_emotions": "[\'neutral\']"}',
 '{"utterance_emotions": "[\'neutral\']"}',
 '{"utterance_emotions": "[\'neutral\']"}',
 '{"utterance_emotions": "[\'neutral\']"}',
 '{"utterance_emotions": "[\'joy\']"}',
 '{"utterance_emotions": "[\'fear\', \'surprise\']"}',
 '{"utterance_emotions": "[\'neutral\']"}',
 '{"utterance_emotions": "[\'joy\']"}',
 '{"utterance_emotions": "[\'neutral\']"}',
 '{"utterance_emotions": "[\'neutral\']"}',
 '{"utterance_emotions": "[\'neutral\']"}',
 '{"utterance_emotions": "[\'sadness\']"}',
 '{"utterance_emotions": "[\'anger\']"}',
 '{"utterance_emotions": "[\'anger\', \'disgust\']"}',
 '{"utterance_emotions": "[\'joy\', \'surprise\']"}',
 '{"utterance_emotions": "[\'fear\', \'anger\', \'surprise\']"}',
 '{"utterance_emotions": "[\'neutral\']"}',
 '{"utterance_emotions": "[\'anger\', \'sadness\']"}',
 '{"utterance_emotions": "[\'anger\', \'sadness\']"}',
 '{"utterance_emotions": "[\'anger\', \'sadness\']"}',

In [155]:
#predictions = [prediction.split("###")[0] for prediction in predictions]

In [9]:
len(predictions)

1776

In [128]:
# del predictions[502]
# del predictions[516]
# del predictions[517]
# del predictions[992]
# del predictions[993]
# del predictions[995]
# del predictions[1671]
# del predictions[1672]
# del predictions[1674]
# del predictions[1677]

# del grounds[502]
# del grounds[516]
# del grounds[517]
# del grounds[992]
# del grounds[993]
# del grounds[995]
# del grounds[1671]
# del grounds[1672]
# del grounds[1674]
# del grounds[1677]

In [10]:
preds = []

    #for output in outputs_l:
for i, prediction in enumerate(predictions):
    try:
        # Use json.loads to safely parse the JSON-like string
        parsed_prediction = json.loads(prediction)
        # Append the values of the parsed prediction to preds
        preds.append(parsed_prediction['utterance_emotions'])
        
    except json.JSONDecodeError as e:
        print(f"Error decoding prediction at index {i}: {e}")
        
    except KeyError as e:
        print(f"Key error at index {i}: {e}")
        
    except Exception as e:
        print(f"Unexpected error at index {i}: {e}")

In [11]:
len(grounds), len(predictions)

(1776, 1776)

In [12]:
grounds

[['surprise', 'joy'],
 ['joy'],
 ['surprise', 'joy'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['surprise'],
 ['joy'],
 ['joy'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['anger', 'disgust'],
 ['anger', 'disgust'],
 ['neutral'],
 ['sadness'],
 ['sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['fear', 'surprise'],
 ['surprise'],
 ['joy'],
 ['anger', 'surprise'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['surprise', 'joy'],
 ['fear', 'sadness'],
 ['fear', 'sadness'],
 ['fear', 'surprise'],
 ['anger', 'disgust'],
 ['anger', 'disgust'],
 ['anger', 'disgust'],
 ['fear', 'sadness'],
 ['fear', 'sadness', 'surprise'],
 ['sadness'],
 ['sadness'],
 ['fear', 'sadness'],
 ['sadness', 'surprise'],
 ['sadness', 'surprise'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['anger'],
 ['anger', 'disgust'],
 ['joy'],
 ['joy'],
 ['surprise', 'joy'],
 ['surprise', 'joy'],
 ['anger', 'surprise'],
 ['anger', 'surprise'],
 ['neutral'],
 ['joy'],
 ['joy'],
 ['neutral'

In [160]:
preds

["['neutral']",
 "['neutral']",
 "['joy']",
 "['joy']",
 "['joy', 'neutral']",
 "['joy']",
 "['fear', 'surprise']",
 "['joy']",
 "['joy']",
 "['joy']",
 "['neutral']",
 "['anger', 'surprise']",
 "['sadness']",
 "['anger']",
 "['anger', 'disgust']",
 "['joy', 'surprise']",
 "['anger', 'fear', 'sadness']",
 "['anger', 'sadness']",
 "['anger', 'sadness']",
 "['anger', 'sadness']",
 "['anger', 'sadness']",
 "['fear', 'surprise']",
 "['neutral']",
 "['fear', 'sadness']",
 "['anger', 'surprise']",
 "['anger', 'fear']",
 "['joy']",
 "['joy']",
 "['anger', 'sadness']",
 "['anger', 'surprise']",
 "['anger', 'surprise']",
 "['sadness', 'surprise']",
 "['joy']",
 "['anger', 'surprise']",
 "['anger', 'disgust']",
 "['fear', 'sadness']",
 "['anger', 'surprise']",
 "['sadness']",
 "['surprise', 'fear']",
 "['anger', 'surprise']",
 "['anger', 'disgust']",
 "['anger', 'sadness']",
 "['anger', 'fear']",
 "['anger', 'fear', 'sadness']",
 "['joy']",
 "['anger']",
 "['fear', 'surprise']",
 "['anger', 'sad

In [13]:
preds_l = []

for i, pred in enumerate(preds):
    try:
        preds_l.append(ast.literal_eval(str(pred)))
    except ValueError as e:
        print(f"Unexpected error at index {i}: {e}")    

In [14]:
preds_l

[['neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['joy'],
 ['fear', 'surprise'],
 ['neutral'],
 ['joy'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['sadness'],
 ['anger'],
 ['anger', 'disgust'],
 ['joy', 'surprise'],
 ['fear', 'anger', 'surprise'],
 ['neutral'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['anger', 'sadness'],
 ['surprise', 'fear'],
 ['surprise'],
 ['neutral'],
 ['anger', 'surprise'],
 ['neutral'],
 ['joy'],
 ['joy'],
 ['anger', 'sadness'],
 ['anger'],
 ['anger', 'sadness'],
 ['sadness', 'neutral'],
 ['joy'],
 ['neutral'],
 ['anger', 'fear'],
 ['sadness'],
 ['anger'],
 ['sadness'],
 ['surprise', 'neutral'],
 ['surprise'],
 ['anger', 'fear'],
 ['surprise', 'anger'],
 ['frustration', 'fear'],
 ['anger', 'fear'],
 ['joy'],
 ['anger'],
 ['neutral', 'surprise'],
 ['anger', 'fear'],
 ['anger', 'sadness'],
 ['joy'],
 ['joy'],
 ['joy', 'neutral'],
 ['joy'],
 ['anger', 'disgust'],
 ['anger', 'surprise'],
 ['joy', 'surprise'],
 ['joy'],
 ['joy'],
 ['neu

In [15]:
all_labels = ["anger", "surprise", "fear", "disgust", "sadness", "joy", "neutral"]

def read_json_preds(decoded_outputs):
    
    preds = []

    #for output in outputs_l:
    for i, prediction in enumerate(decoded_outputs):
        try:
            # Use json.loads to safely parse the JSON-like string
            parsed_prediction = json.loads(prediction)
            # Append the values of the parsed prediction to preds
            preds.append(parsed_prediction['utterance_emotions'])
            
        except json.JSONDecodeError as e:
            print(f"Error decoding prediction: {i}")
            
    preds = [ast.literal_eval(item) for item in preds]    
            
    return preds



def labels_to_binary_matrix(label_list, all_labels):
    binary_matrix = np.zeros((len(label_list), len(all_labels)))
    
    for i, labels in enumerate(label_list):
        for label in labels:
            if label in all_labels:
                binary_matrix[i][all_labels.index(label)] = 1
                
    return binary_matrix

def opposite(component_type):

    if component_type == "anger":
        return "surprise"
    elif component_type == "disgust":
        return "joy"
    elif component_type == "fear":
        return "sadness"
    elif component_type == "sadness":
        return "anger"
    elif component_type == "surprise":
        return "disgust"
    elif component_type == "joy":
        return "fear"
    elif component_type == "Neutral":
        return "sadness"
    

def harmonize_preds(grounds, preds):

    l1, l2 = len(preds), len(grounds)
    if l1 < l2:
        diff = l2 - l1
        preds = preds + [opposite(x) for x in grounds[l1:]]
    else:
        preds = preds[:l2]
        
    return preds 

def post_process_icl(grounds, preds):

    for i,(x,y) in enumerate(zip(grounds, preds)):
        
        if len(x) != len(y):
            
            preds[i] = harmonize_preds(x, y)

    true_matrix = labels_to_binary_matrix(grounds, all_labels)
    predicted_matrix = labels_to_binary_matrix(preds, all_labels)

    return true_matrix, predicted_matrix

In [16]:
true_matrix, predicted_matrix = post_process_icl(grounds, preds_l)

In [17]:
print(classification_report(true_matrix, predicted_matrix, target_names=all_labels, digits=3))

              precision    recall  f1-score   support

       anger      0.648     0.477     0.550       614
    surprise      0.660     0.527     0.586       486
        fear      0.400     0.435     0.416       407
     disgust      0.118     0.235     0.157        85
     sadness      0.416     0.352     0.381       347
         joy      0.547     0.503     0.524       429
     neutral      0.136     0.295     0.186       129

   micro avg      0.464     0.449     0.456      2497
   macro avg      0.418     0.403     0.400      2497
weighted avg      0.516     0.449     0.475      2497
 samples avg      0.447     0.440     0.442      2497



/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
classification_file = Path("/Utilisateurs/umushtaq/emotion_analysis_comics/incontext_learning/results/icl_mistral-7b-instruct-v0.3-bnb-4bit") / f"classification_report_{10}.pickle"

with classification_file.open('wb') as fh:
    
    pickle.dump(classification_report(true_matrix, predicted_matrix, target_names=all_labels, output_dict=True), fh)

/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
